Телеграм-бот – помощник по проверке сочинений для ЕГЭ по русскому по критериям 2025 года

Библиотеки:

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import class_weight
from safetensors.torch import load_file
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import nest_asyncio
import os


c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.sparse import csr_matrix, issparse


Обучение: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CRITERIA = [f"K{i}" for i in range(1, 11)]
NUM_CLASSES = [2, 4, 3, 2, 3, 2, 4, 4, 4, 4]

df = pd.read_csv("sochineniya_score.csv", sep=';')

print(df.head())

data_dict = {
    "essay": df["essay"].tolist(),
    "labels": df[CRITERIA].values.tolist()
}

dataset = Dataset.from_dict(data_dict)

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

def preprocess_function(examples):
    encodings = tokenizer(examples["essay"], truncation=True, padding=False)
    encodings["labels"] = examples["labels"]
    return encodings

dataset = dataset.map(preprocess_function, batched=True)

split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
val_dataset = split["test"]

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_class_weights_safe(y, num_classes, device='cpu'):
    unique_classes = np.unique(y)
    weights_existing = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=unique_classes,
        y=y
    )
    weights_full = np.ones(num_classes, dtype=np.float32)
    for cls, w in zip(unique_classes, weights_existing):
        weights_full[cls] = w
    weights_tensor = torch.tensor(weights_full, dtype=torch.float32, device=device)
    return weights_tensor

class_weights_per_criterion = []
for i, crit in enumerate(CRITERIA):
    num_classes = NUM_CLASSES[i]
    labels = df[crit].values.astype(int) 
    weights_tensor = compute_class_weights_safe(labels, num_classes, device=device)
    class_weights_per_criterion.append(weights_tensor)

def set_zeros_for_short_essays(essays, preds):
    for i, essay in enumerate(essays):
        if len(essay.split()) < 150:
            preds[i] = np.zeros(preds.shape[1], dtype=int)
    return preds

class MultiHeadRubertModel(nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.heads = nn.ModuleList([
            nn.Linear(self.bert.config.hidden_size, num_class) for num_class in NUM_CLASSES
        ])
        self.class_weights = class_weights

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        logits_list = [head(pooled_output) for head in self.heads]

        if labels is not None:
            total_loss = 0
            for i, logits in enumerate(logits_list):
                loss_fct = nn.CrossEntropyLoss(weight=self.class_weights[i])
                total_loss += loss_fct(logits, labels[:, i])
            loss = total_loss / len(self.heads)
            return {"loss": loss, "logits": logits_list}
        else:
            return {"logits": logits_list}

model = MultiHeadRubertModel(class_weights=class_weights_per_criterion).to(device)

def compute_metrics(eval_pred):
    logits_list, labels = eval_pred
    preds = []
    for i in range(len(NUM_CLASSES)):
        preds_i = np.argmax(logits_list[i], axis=1)
        preds.append(preds_i)
    preds = np.stack(preds, axis=1)

    acc_list, f1_list = [], []
    for i in range(len(NUM_CLASSES)):
        acc = accuracy_score(labels[:, i], preds[:, i])
        f1 = f1_score(labels[:, i], preds[:, i], average="macro")
        acc_list.append(acc)
        f1_list.append(f1)

    return {
        "accuracy_mean": np.mean(acc_list),
        "f1_macro_mean": np.mean(f1_list),
        "accuracy_per_criterion": acc_list,
        "f1_macro_per_criterion": f1_list,
    }

training_args = TrainingArguments(
    output_dir="output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro_mean",
    greater_is_better=True,
    save_total_limit=2,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train()

eval_result = trainer.evaluate()
print("Evaluation results:", eval_result)

pred_output = trainer.predict(val_dataset)

logits_list = pred_output.predictions

if isinstance(logits_list, list):
    preds_per_criterion = [np.argmax(logits, axis=1) for logits in logits_list]
    preds = np.stack(preds_per_criterion, axis=1)
else:
    print("Warning: pred_output.predictions is not list.")
    preds = None  

essays = val_dataset["essay"]
preds = set_zeros_for_short_essays(essays, preds)

print("Predictions shape:", preds.shape)


c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.sparse import csr_matrix, issparse


                                               essay  K1  K2  K3  K4  K5  K6  \
0  В чём состоят цели творчества художника? Именн...   1   3   2   1   2   1   
1  А.П. Чехов в предложенном фрагменте пьесы «Дяд...   0   0   0   1   1   1   
2  В своем произведении Ю.К. Олеша задается вопро...   1   3   2   1   2   1   
3  Настоящая критика… Какой она должна быть? Имен...   1   3   2   1   2   1   
4  В центре внимания А.С. Новикова-Прибоя находит...   1   3   2   1   2   1   

   K7  K8  K9  K10  total_score  
0   3   3   3    3           22  
1   1   2   3    3           12  
2   3   2   2    3           20  
3   3   3   3    3           22  
4   3   3   3    3           22  


Map: 100%|██████████| 233/233 [00:00<00:00, 3135.41 examples/s]
C:\Users\taike\AppData\Local\Temp\ipykernel_64348\1560556830.py:148: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy Mean,F1 Macro Mean,Accuracy Per Criterion,F1 Macro Per Criterion
1,No log,1.071264,0.612766,0.369377,"[0.9787234042553191, 0.9787234042553191, 0.9148936170212766, 0.02127659574468085, 0.0, 0.9787234042553191, 0.0, 0.2765957446808511, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4777777777777778, 0.020833333333333332, 0.0, 0.4946236559139785, 0.0, 0.21666666666666667, 1.0, 0.4946236559139785]"
2,No log,1.055602,0.644681,0.384771,"[0.9787234042553191, 0.9787234042553191, 0.9361702127659575, 0.02127659574468085, 0.0, 0.9787234042553191, 0.0, 0.574468085106383, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4835164835164835, 0.020833333333333332, 0.0, 0.4946236559139785, 0.0, 0.36486486486486486, 1.0, 0.4946236559139785]"
3,No log,1.039974,0.661702,0.391649,"[0.9787234042553191, 0.9787234042553191, 0.9361702127659575, 0.02127659574468085, 0.02127659574468085, 0.9787234042553191, 0.0, 0.723404255319149, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4835164835164835, 0.020833333333333332, 0.013888888888888888, 0.4946236559139785, 0.0, 0.41975308641975306, 1.0, 0.4946236559139785]"
4,No log,1.026700,0.682979,0.401099,"[0.9787234042553191, 0.9787234042553191, 0.9574468085106383, 0.02127659574468085, 0.1276595744680851, 0.9787234042553191, 0.0, 0.8085106382978723, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4891304347826087, 0.020833333333333332, 0.07547169811320754, 0.4946236559139785, 0.0, 0.4470588235294118, 1.0, 0.4946236559139785]"
5,No log,1.015461,0.719149,0.419149,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.02127659574468085, 0.2553191489361702, 0.9787234042553191, 0.0851063829787234, 0.9361702127659575, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.020833333333333332, 0.13559322033898305, 0.4946236559139785, 0.0784313725490196, 0.4835164835164835, 1.0, 0.4946236559139785]"
6,No log,1.004193,0.751064,0.437433,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.0851063829787234, 0.425531914893617, 0.9787234042553191, 0.14893617021276595, 0.9574468085106383, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.08344671201814059, 0.1990049751243781, 0.4946236559139785, 0.12962962962962962, 0.4891304347826087, 1.0, 0.4946236559139785]"
7,No log,0.994631,0.787234,0.460558,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.2127659574468085, 0.48936170212765956, 0.9787234042553191, 0.2978723404255319, 0.9787234042553191, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.18927738927738927, 0.21904761904761905, 0.4946236559139785, 0.22950819672131148, 0.4946236559139785, 1.0, 0.4946236559139785]"
8,No log,0.986550,0.846809,0.541051,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.46808510638297873, 0.5957446808510638, 0.9787234042553191, 0.5106382978723404, 1.0, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.3504698728579326, 0.2488888888888889, 0.4946236559139785, 0.3380281690140845, 1.0, 1.0, 0.4946236559139785]"
9,No log,0.979505,0.885106,0.560001,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.7446808510638298, 0.5957446808510638, 0.9787234042553191, 0.6170212765957447, 1.0, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.49642857142857144, 0.2488888888888889, 0.4946236559139785, 0.3815789473684211, 1.0, 1.0, 0.4946236559139785]"
10,No log,0.973502,0.919149,0.589549,"[0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.8723404255319149, 0.7021276595744681, 0.9787234042553191, 0.723404255319149, 1.0, 1.0, 0.9787234042553191]","[0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.5901162790697674, 0.4125, 0.4946236559139785, 0.41975308641975306, 1.0, 1.0, 0.4946236559139785]"


c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warn

Evaluation results: {'eval_loss': 0.9688538908958435, 'eval_accuracy_mean': 0.9404255319148935, 'eval_f1_macro_mean': 0.6168262566177594, 'eval_accuracy_per_criterion': [0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.9787234042553191, 0.7446808510638298, 0.9787234042553191, 0.7872340425531915, 1.0, 1.0, 0.9787234042553191], 'eval_f1_macro_per_criterion': [0.4946236559139785, 0.4946236559139785, 0.4946236559139785, 0.8278388278388278, 0.4268292682926829, 0.4946236559139785, 0.44047619047619047, 1.0, 1.0, 0.4946236559139785], 'eval_runtime': 0.4542, 'eval_samples_per_second': 103.47, 'eval_steps_per_second': 6.604, 'epoch': 13.0}


c:\Users\taike\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predictions shape: (47, 10)


Телеграм-бот:

In [ ]:
NUM_CLASSES = [2, 4, 3, 2, 3, 2, 4, 4, 4, 4]
MAX_LENGTH = 512

BOT_TOKEN = "7687048119:AAGzDXtsD405uqd13Fh5mHRP4KztkVGtXDw"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CRITERIA = [f"K{i}" for i in range(1, 11)]

explanations_dict = {
    "K1": {0: "Позиция автора не сформулирована.", 1: "Позиция автора сформулирована верно."},
    "K2": {0: "Позиция автора не сформулирована, или не приведены примеры-иллюстрации, или не даны пояснения к ним.",
           1: "Позиция автора (рассказчика) по указанной проблеме исходного текста прокомментирована с опорой на исходный текст. Приведён 1 пример-иллюстрация из прочитанного текста, важный для понимания позиции автора (рассказчика) по указанной проблеме исходного текста. Дано пояснение к этому примеру-иллюстрации.",
           2: "Позиция автора (рассказчика) по указанной проблеме исходного текста прокомментирована с опорой на исходный текст. Приведено 2 примера-иллюстрации из прочитанного текста, важных для понимания позиции автора (рассказчика) по указанной проблеме исходного текста. Дано пояснение к каждому из примеров-иллюстраций.Смысловая связь между приведёнными примерами-иллюстрациями не указана, или не дано её пояснение, или дано неверное пояснение.",
           3: "Позиция автора (рассказчика) по указанной проблеме исходного текста прокомментирована с опорой на исходный текст. Приведено 2 примера-иллюстрации из прочитанного текста, важных для понимания позиции автора (рассказчика) по указанной проблеме исходного текста. Дано пояснение к каждому из примеров-иллюстраций. Указана смысловая связь между приведёнными примерами-иллюстрациями. Дано пояснение к ней."},
    "K3": {0: "Собственное отношение к позиции автора (рассказчика) по указанной проблеме исходного текста не сформулировано и не обосновано.",
           1: "Собственное отношение к позиции автора (рассказчика) по указанной проблеме исходного текста сформулировано и обосновано. Пример-аргумент не приведён.",
           2: "Собственное отношение к позиции автора (рассказчика) по указанной проблеме исходного текста сформулировано и обосновано. Приведён пример-аргумент."},
    "K4": {0: "Допущена одна фактическая ошибка или более.", 1: "Фактические ошибки отсутствуют."},
    "K5": {0: "Допущены две логические ошибки или более.", 1: "Допущена одна логическая ошибка.", 2: "Логические ошибки отсутствуют"},
    "K6": {0: "В работе приводятся примеры экстремистских и/или иных запрещённых к производству и распространению среди несовершеннолетних материалов / социально неприемлемого поведения людей / имеются высказывания, нарушающие законодательство Российской Федерации.", 1: "Этические ошибки отсутствуют."},
    "K7": {0: "Допущены пять орфографических ошибок или более", 1: "Допущены три-четыре орфографические ошибки.", 2: "Допущены одна-две орфографические ошибки.", 3: "Орфографических ошибок нет."},
    "K8": {0: "Допущены пять пунктуационных ошибок или более", 1: "Допущены три-четыре пунктуационные ошибки.", 2: "Допущены одна-две пунктуационные ошибки.", 3: "Пунктуационных ошибок нет."},
    "K9": {0: "Допущены пять грамматических ошибок или более", 1: "Допущены три-четыре грамматические ошибки.", 2: "Допущены одна-две грамматические ошибки.", 3: "Грамматических ошибок нет."},
    "K10": {0: "Допущены пять речевых ошибок или более", 1: "Допущены три-четыре речевые ошибки.", 2: "Допущены одна-две речевые ошибки.", 3: "Речевых ошибок нет."}
}


class MultiHeadRubertModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.heads = nn.ModuleList([
            nn.Linear(self.bert.config.hidden_size, num_class) for num_class in NUM_CLASSES
        ])

    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output 
        logits_list = [head(pooled_output) for head in self.heads]
        return logits_list

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiHeadRubertModel()

checkpoints_path = "C:/Users/taike/OneDrive/Рабочий стол/Final project/output"
checkpoint_dirs = [d for d in os.listdir(checkpoints_path) if d.startswith("checkpoint-")]
latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(x.split("-")[1]))
model_path = os.path.join(checkpoints_path, latest_checkpoint, "model.safetensors")

state_dict = load_file(model_path, device=DEVICE)

model.load_state_dict(state_dict)
model.to(torch.device(DEVICE)) 
model.eval()


def predict_and_explain(text: str):
    inputs = tokenizer(
        text,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    ).to(DEVICE)

    with torch.no_grad():
        logits_list = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

    preds = [torch.argmax(logits, dim=-1).item() for logits in logits_list]

    explanations = []
    for i, crit in enumerate(CRITERIA):
        pred_class = preds[i]
        explanation = explanations_dict.get(crit, {}).get(pred_class, f"Балл {pred_class} по критерию {crit}")
        explanations.append((crit, pred_class, explanation))

    total_score = sum(preds)
    return preds, explanations, total_score


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Привет! Пришли мне текст сочинения, и я оценю его по 10 критериям.")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text
    if not text or text.strip() == "":
        await update.message.reply_text("Пожалуйста, пришли текст сочинения для оценки.")
        return

    if len(text.split()) < 150:
        await update.message.reply_text("Итоговый балл: 0. В сочинении меньше 150 слов. Нужно увеличить его объём.")
        return

    preds, explanations, total_score = predict_and_explain(text)

    reply = [f"Итоговый балл: {total_score}\n"]
    for crit, pred, expl in explanations:
        reply.append(f"{crit}: {pred}\n{expl}\n")

    await update.message.reply_text("\n".join(reply))

async def main():
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Бот запущен. Ожидаю сообщения...")
    await app.run_polling()

import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    asyncio.run(main())


KeyboardInterrupt: 